In [1]:
import pandas as pd
import numpy as np
import requests
import json

### 1. We start off by getting the matches from the pickled files

In [2]:
practice_data_2000_3000 = pd.read_pickle("../data/practice_data_2000_3000.pkl")
practice_data_3000_4000 = pd.read_pickle("../data/practice_data_3000_4000.pkl")
practice_data_no_limit = pd.read_pickle("../data/practice_data_no_limit.pkl")

test_data_2000_3000 = pd.read_pickle("../data/test_data_2000_3000.pkl")
test_data_3000_4000 = pd.read_pickle("../data/test_data_3000_4000.pkl")
test_data_no_limit = pd.read_pickle("../data/test_data_no_limit.pkl")

### 2. Let's start inspecting our dataframe

**We do have to be mindful that all matches are duplicated 10 times, since there is a row for each picked hero in each match**

In [3]:
practice_data_2000_3000

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,player_slot,hero_id
0,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,132,47
1,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,131,8
2,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,130,21
3,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,129,84
4,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,128,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,5504961700,4615512732,True,1594245043,1875,2646,3,7,22,46,7,203,3,8
999996,5504961700,4615512732,True,1594245043,1875,2646,3,7,22,46,7,203,2,22
999997,5504961700,4615512732,True,1594245043,1875,2646,3,7,22,46,7,203,1,108
999998,5504961700,4615512732,True,1594245043,1875,2646,3,7,22,46,7,203,0,30


**As seen below, all the datatypes seem to come in the right format**

In [4]:
practice_data_2000_3000.dtypes

match_id         int64
match_seq_num    int64
radiant_win       bool
start_time       int64
duration         int64
avg_mmr          int64
num_mmr          int64
lobby_type       int64
game_mode        int64
avg_rank_tier    int64
num_rank_tier    int64
cluster          int64
player_slot      int64
hero_id          int64
dtype: object

**There are zero null values, as this was handled in the SQL query to the API**

In [5]:
practice_data_2000_3000.isna().sum()

match_id         0
match_seq_num    0
radiant_win      0
start_time       0
duration         0
avg_mmr          0
num_mmr          0
lobby_type       0
game_mode        0
avg_rank_tier    0
num_rank_tier    0
cluster          0
player_slot      0
hero_id          0
dtype: int64

## 3. We now want to clean our dataframe to a usable format

**We want to go from this table:**

| match_id | ... | player_slot | hero_id |
|------|------|------|------|
|5513437212|...|Dire|84|
|5513437212|...|Dire|8|
|5513437212|...|Dire|39|
|5513437212|...|Dire|29|
|5513437212|...|Dire|80|
|5513437212|...|Radiant|46|
|5513437212|...|Radiant|75|
|5513437212|...|Radiant|96|
|5513437212|...|Radiant|4|
|5513437212|...|Radiant|25|

**To this table:**

| match_id | ... | radiant1 | radiant2 | radiant3 | radiant4 | radiant5 |  dire1 | dire2 | dire3 | dire4 | dire5 |
|------|------|------|------|------|------|------|------|------|------|------|------|
|5513437212|...|46|75|96|4|25|84|8|39|29|80|

**We write a function to do this:**

In [6]:
def table_cleaner(dataframe):
    # 0-4: Radiant team
    # 128-132: Dire team
    #If the player slot is between 0-4 the player is on the radiant team
    #if the player slot is between 128-132 the player is on the dire team
    slot_dict = {0: "radiant_1",
                 1: "radiant_2",
                 2: "radiant_3",
                 3: "radiant_4",
                 4: "radiant_5",
                 128: "dire_1",
                 129: "dire_2",
                 130: "dire_3",
                 131: "dire_4",
                 132: "dire_5"}
    
    # We drop the columns we don't need
    dataframe.drop(["num_mmr", "lobby_type", "game_mode", "avg_rank_tier", "num_rank_tier"], axis=1, inplace=True)
    
    # We create columns for each player slot to make the df horisontal rather than vertical
    for slot in slot_dict:
        dataframe.loc[dataframe["player_slot"] == slot, slot_dict[slot]] = 1
    
    # We fill the NaN values with 0 and converts the columns to integers rather than floats
    dataframe = dataframe.fillna(0).astype(int)
    
    # We now multiply the binary columns with the `hero_id` to get the hero id out there. 
    # If the column is 0, it will stay 0
    # for the radiant team
    dataframe["radiant_1"] = dataframe["hero_id"] * dataframe["radiant_1"]
    dataframe["radiant_2"] = dataframe["hero_id"] * dataframe["radiant_2"]
    dataframe["radiant_3"] = dataframe["hero_id"] * dataframe["radiant_3"]
    dataframe["radiant_4"] = dataframe["hero_id"] * dataframe["radiant_4"]
    dataframe["radiant_5"] = dataframe["hero_id"] * dataframe["radiant_5"]
    # for the dire team
    dataframe["dire_1"] = dataframe["hero_id"] * dataframe["dire_1"]
    dataframe["dire_2"] = dataframe["hero_id"] * dataframe["dire_2"]
    dataframe["dire_3"] = dataframe["hero_id"] * dataframe["dire_3"]
    dataframe["dire_4"] = dataframe["hero_id"] * dataframe["dire_4"]
    dataframe["dire_5"] = dataframe["hero_id"] * dataframe["dire_5"]
    
    # We don't need the "hero_id" or "player_slot" columns anymore, so we drop them
    dataframe.drop(["hero_id", "player_slot"], axis=1, inplace=True)
    
    # We now group the matches to only have one row per match_id
    dataframe = dataframe.groupby(["match_id",
                                   "match_seq_num",
                                   "radiant_win",
                                   "start_time",
                                   "duration",
                                   "avg_mmr"]).agg({"radiant_1": "sum",
                                                    "radiant_2": "sum",
                                                    "radiant_3": "sum",
                                                    "radiant_4": "sum",
                                                    "radiant_5": "sum",
                                                    "dire_1": "sum",
                                                    "dire_2": "sum",
                                                    "dire_3": "sum",
                                                    "dire_4": "sum",
                                                    "dire_5": "sum"}).reset_index()
    
    # We want to make sure "radiant_win" stays a boolean and does not get converted to an int
    dataframe["radiant_win"] = dataframe["radiant_win"].astype(bool)
    
    dataframe = dataframe.replace(0, np.nan)
    dataframe = dataframe.dropna()
    dataframe[["radiant_1",
               "radiant_2",
               "radiant_3",
               "radiant_4",
               "radiant_5",
               "dire_1",
               "dire_2",
               "dire_3",
               "dire_4",
               "dire_5"]] = dataframe[["radiant_1",
                                       "radiant_2",
                                       "radiant_3",
                                       "radiant_4",
                                       "radiant_5",
                                       "dire_1",
                                       "dire_2",
                                       "dire_3",
                                       "dire_4",
                                       "dire_5"]].astype(int)
    
    # We return the processed dataframe
    return dataframe.reset_index(drop=True)

## 4. Creating winrate pivot tables for synergy and counter


**We want to create pivot tables that can compute the winrate of synergy and counter in each hero pair**
**The synergy pivot table would look like this:**

| hero_id | 1 | 2 | 3 | 4 | ... | 128 | 129 |
|------|------|------|------|------|------|------|------|
| **1** |NaN|0.53|0.42|0.52|...|0.63|0.57|
| **2** |0.53|NaN|0.51|0.53|...|0.53|0.53|
| **3** |0.42|0.51|NaN|0.55|...|0.53|0.53|
| **4** |0.52|0.53|0.55|NaN|...|0.53|0.51|
| **5** |0.56|0.53|0.53|0.53|...|0.53|0.58|
|...|...|...|...|...|...|...|...|
| **128** |0.63|0.53|0.53|0.53|...|NaN|0.51|
| **129** |0.57|0.53|0.51|0.53|...|0.51|NaN|

In this case, the pivot table is mirrored as it is equally good/bad when the heroes are on the same team. 

**The counter pivot table would look like this:**

| hero_id | 1 | 2 | 3 | 4 | ... | 128 | 129 |
|------|------|------|------|------|------|------|------|
| **1** |NaN|0.47|0.58|0.48|...|0.37|0.43|
| **2** |0.53|NaN|0.49|0.47|...|0.47|0.47|
| **3** |0.42|0.51|NaN|0.45|...|0.47|0.49|
| **4** |0.52|0.53|0.55|NaN|...|0.47|0.47|
| **5** |0.56|0.53|0.53|0.53|...|0.53|0.58|
|...|...|...|...|...|...|...|...|
| **128** |0.63|0.53|0.53|0.53|...|NaN|0.49|
| **129** |0.57|0.53|0.51|0.53|...|0.51|NaN|

When it comes to the counter pivot table, the winrate will always be 1 minus the winrate for a pair. 
For instance hero 129 has a winrate of 0.63 vs hero 13. This means that hero 13 has a winrate of 0.37 vs hero 129

**Now we make a function that splits the dataset in radiant wins and dire wins**

In [7]:
# The split the dataframe into two, one when radiant won, and one when dire won
def winning_side_splitter(dataframe):
    # Dataframe containing only radiant wins
    df_matches_radiant = dataframe.loc[dataframe["radiant_win"] == 1].reset_index()
    # Dataframe containing only dire wins
    df_matches_dire = dataframe.loc[dataframe["radiant_win"] == 0].reset_index()
    return df_matches_radiant, df_matches_dire

We want to create a two pivot tables:

* Synergy winrate (contains the winrate-synergy between two heroes)
* Counter winrate (contains the winrate-counter between two heroes)

To get that, we need the following pivot tables:

* radiant_synergy_win
* radiant_synergy_loss
* radiant_opponent_win
* radiant_opponent_loss
* dire_synergy_win
* dire_synergy_loss
* dire_opponent_win
* dire_opponent_loss

**To get the synergy winrate:**

synergy_wins = dire_synergy_wins + radiant_synergy_wins

synergy_losses = dire_synergy_loss + radiant_synergy_loss

synergy_winrate = synergy_wins / (synergy_wins + synergy_losses)


**The same goes for the counter winrate:**

counter_wins = dire_counter_wins + radiant_counter_wins

counter_losses = dire_counter_losses + radiant_counter_losses

counter_winrate = counter_wins / (counter_wins + counter_losses)

### We start of by creating the functions for the synergy pivot table
**Now we create the function that creates the 20 combined synergy pivot tables, to later add together**

In [8]:
def pair_pivots_synergy(matches_dataframe, permutations):
    # We declared the list to append all the pivot_tables on
    list_of_dfs = []
    # We declared the name count to keep track of the pivot tables
    name_count = 0
    # We declare an empty dataframe to input the pivot tables
    pivot = {}
    
    # Now we loop through the columns in the team except its own columns
    for permutation1 in range(len(permutations)):
        # We loop through the perpendicular columns
        for permutation2 in range(len(permutations)):
            # We make sure not to include the column in question
            if permutations[permutation1] != permutations[permutation2]:
                # Increment the name_count by one
                name_count+=1
                
                #We create a pivot table for each hero matchup that happened where the given side won
                pivot = pd.pivot_table(matches_dataframe
                                 # We group by the hero1 with hero2
                                 .groupby([permutations[permutation1], permutations[permutation2]])
                                 # We count the occurences of each occurence
                                 .agg(count=(permutations[permutation2], "count"))
                                 # We reset the index
                                 .reset_index()
                                 # And rename the columns from "direX" and "direY" to "hero_1" and "hero_2"
                                 # to be able to add them together later on column names
                                 .rename(columns={permutations[permutation1]: "hero_1", permutations[permutation2]: "hero_2", "count": f"count{name_count}"}),
                                     index="hero_1",
                                     columns="hero_2",
                                     values=f"count{name_count}",
                                     dropna=False,
                                     fill_value=0)
                
                # We append the pivot table to a list of dataframes for later use
                list_of_dfs.append(pivot)
    
    return list_of_dfs

**Now we combine the pivot tables into one**

In [9]:
# From the pair_pivots function we are returning 20 dataframes in a list. 
# we add them together in to one with the "pivot_combiner"
def pivot_combiner(list_of_pivots):
    frame1 = list_of_pivots[0]
    for frame in range(len(list_of_pivots)):
        if frame != 0:
            frame1 = frame1 + list_of_pivots[frame]
    return frame1

**Now we create a wrapper to calculate the synergy winrate for each hero**

In [10]:
# The synergy_wrapper takes the dataframe that contains the match data with the radiant_1 - dire_5 in the columns
# and creates a synergy_pivot table based on that. The table will be used later to compute the winrate of certain
# pair_matchups
def synergy_wrapper(dataframe):
    # We pass the columns statically, as they will never change for a dota match
    permutations1 = ["radiant_1", "radiant_2", "radiant_3", "radiant_4", "radiant_5"]
    permutations2 = ["dire_1", "dire_2", "dire_3", "dire_4", "dire_5"]
    
    # We use unpacking to return the dfs where dire won and where radiant won
    df_matches_radiant, df_matches_dire = winning_side_splitter(dataframe)
    
    # We create the 20 pivot tables that contains all the columns win pair combinations for dire
    dire_synergy_wins = pivot_combiner(pair_pivots_synergy(df_matches_dire, permutations2))
    
    # We create the 20 pivot tables that contains all the columns loss pair combinations for dire
    dire_synergy_losses = pivot_combiner(pair_pivots_synergy(df_matches_radiant, permutations2))
    
    # We create the 20 pivot tables that contains all the columns win pair combinations for radiant
    radiant_synergy_wins = pivot_combiner(pair_pivots_synergy(df_matches_radiant, permutations1))
    
    # We create the 20 pivot tables that contains all the columns loss pair combinations for radiant
    radiant_synergy_losses = pivot_combiner(pair_pivots_synergy(df_matches_dire, permutations1))
    
    # We compute the total amount of synergy wins
    synergy_wins = dire_synergy_wins + radiant_synergy_wins
    
    # We compute the total amount of synergy losses
    synergy_losses = dire_synergy_losses + radiant_synergy_losses
    
    # We compute the synergy_winrate
    synergy_winrate = synergy_wins.div((synergy_wins.add(synergy_losses)))
    
    # We return the synergy_winrate pivot table
    return round(synergy_winrate,2)

### We will now create the functions to calculate the counter-winrate of the heroes on each team

**We now have the function to compute the synergy of two heroes on the same team.**

We want to create the same pivot tables as seen in the previous section

In [11]:
def pair_pivots_counter(matches_dataframe, permutations1, permutations2):
    # We declare the list to append all the pivot_tables on
    list_of_dfs = []
    # We declare the name count to keep track of the pivot tables
    name_count = 0
    # We declare an empty dataframe to input the pivot tables
    pivot = {}
    
    # Now we loop through the columns in the team
    for permutation1 in range(len(permutations1)):
        # We loop through the perpendicular columns on the opponent team
        for permutation2 in range(len(permutations2)):
            # Increment the name_count by one
            name_count+=1
            
            #We create a pivot table for each hero matchup that happened where the given side won
            pivot = pd.pivot_table(matches_dataframe
                             # We group by the hero1 with hero2
                             .groupby([permutations1[permutation1], permutations2[permutation2]])
                             # We count the occurences of each matchup
                             .agg(count=(permutations2[permutation2], "count"))
                             # We reset the index
                             .reset_index()
                             # And rename the columns from "radiantX" and "direY" to "hero_1" and "hero_2"
                             # to be able to add them together later on column names
                             .rename(columns={permutations1[permutation1]: "hero_1", permutations2[permutation2]: "hero_2", "count": f"count{name_count}"}),
                                 index="hero_1",
                                 columns="hero_2",
                                 values=f"count{name_count}",
                                 dropna=False,
                                 fill_value=0)
            
            # We append the pivot table to a list of dataframes for later use
            list_of_dfs.append(pivot)
    
    return list_of_dfs

**Now we wrap this in a `counter_wrapper` to get our counter-pivot**

In [12]:
def counter_wrapper(dataframe):
    # We pass the columns statically, as they will never change for a dota match
    permutations1 = ["radiant_1", "radiant_2", "radiant_3", "radiant_4", "radiant_5"]
    permutations2 = ["dire_1", "dire_2", "dire_3", "dire_4", "dire_5"]
    
    # We use unpacking to return the dfs where dire won and where radiant won
    df_matches_radiant, df_matches_dire = winning_side_splitter(dataframe)
    
    # We create the 25 pivot tables that contains all the columns win counter combinations for dire vs radiant
    dire_counter_wins = pivot_combiner(pair_pivots_counter(df_matches_dire, permutations2, permutations1))
    
    # We create the 25 pivot tables that contains all the columns loss counter combinations for dire vs radiant
    dire_counter_losses = pivot_combiner(pair_pivots_counter(df_matches_radiant, permutations2, permutations1))
    
    # We create the 25 pivot tables that contains all the columns win counter combinations for radiant vs dire
    radiant_counter_wins = pivot_combiner(pair_pivots_counter(df_matches_radiant, permutations1, permutations2))
    
    # We create the 25 pivot tables that contains all the columns loss counter combinations for radiant vs dire
    radiant_counter_losses = pivot_combiner(pair_pivots_counter(df_matches_dire, permutations1, permutations2))
    
    # We compute the total amount of counter wins
    counter_wins = dire_counter_wins + radiant_counter_wins
    
    # We compute the total amount of counter losses
    counter_losses = dire_counter_losses + radiant_counter_losses
    
    # We compute the counter_winrate
    counter_winrate = counter_wins.div((counter_wins.add(counter_losses)))
    
    # We return the counter_winrate pivot table
    return round(counter_winrate,2)

## 5. Instantiate the functions and save as pickled dataframes

**We now instantiate the functions written above**

In [13]:
train_2000_3000 = table_cleaner(practice_data_2000_3000)
train_3000_4000 = table_cleaner(practice_data_3000_4000)
train_no_limit = table_cleaner(practice_data_no_limit)

synergy_pivot_2000_3000 = synergy_wrapper(train_2000_3000)
counter_pivot_2000_3000 = counter_wrapper(train_2000_3000)

synergy_pivot_3000_4000 = synergy_wrapper(train_3000_4000)
counter_pivot_3000_4000 = counter_wrapper(train_3000_4000)

synergy_pivot_no_limit = synergy_wrapper(train_no_limit)
counter_pivot_no_limit = counter_wrapper(train_no_limit)

**We make sure to clean the test data as well**

In [14]:
test_2000_3000 = table_cleaner(test_data_2000_3000)
test_3000_4000 = table_cleaner(test_data_3000_4000)
test_no_limit = table_cleaner(test_data_no_limit)

**And we save it as pickled dataframes for later use**

In [15]:
train_2000_3000.to_pickle("../data/clean_train_2000_3000.pkl")
train_3000_4000.to_pickle("../data/clean_train_3000_4000.pkl")
train_no_limit.to_pickle("../data/clean_train_no_limit.pkl")

test_2000_3000.to_pickle("../data/clean_test_2000_3000.pkl")
test_3000_4000.to_pickle("../data/clean_test_3000_4000.pkl")
test_no_limit.to_pickle("../data/clean_test_no_limit.pkl")

synergy_pivot_2000_3000.to_pickle("../data/2000_3000_synergy_pivot.pkl")
counter_pivot_2000_3000.to_pickle("../data/2000_3000_counter_pivot.pkl")
synergy_pivot_3000_4000.to_pickle("../data/3000_4000_synergy_pivot.pkl")
counter_pivot_3000_4000.to_pickle("../data/3000_4000_counter_pivot.pkl")
synergy_pivot_no_limit.to_pickle("../data/no_limit_synergy_pivot.pkl")
counter_pivot_no_limit.to_pickle("../data/no_limit_counter_pivot.pkl")

## 6. Run below cells to check that tables look correct

In [16]:
pd.read_pickle("../data/df_matches_2000_3000.pkl")

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5
0,5504961700,4615512732,True,1594245043,1875,2646,30,108,22,8,16,126,39,28,42,68
1,5504961819,4615510394,True,1594245052,2055,2321,50,104,44,39,9,41,120,74,105,30
2,5504961908,4615513646,True,1594245062,2380,2729,75,30,7,70,108,51,40,10,112,17
3,5504961915,4615516612,True,1594245056,2638,2383,68,40,1,82,107,6,64,71,46,25
4,5504962018,4615510348,True,1594245071,2116,2275,16,62,44,106,104,120,1,94,35,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,5513436411,4623083608,False,1594771154,2584,2100,36,71,32,26,35,11,126,14,94,62
99995,5513436513,4623074188,True,1594771155,1923,2416,14,35,104,48,30,47,121,8,23,119
99996,5513436514,4623080027,True,1594771159,2398,2682,14,106,104,8,64,90,7,74,67,71
99997,5513436705,4623074422,True,1594771169,2052,2198,25,26,14,96,114,73,40,45,93,47


In [17]:
pd.read_pickle("../data/df_matches_3000_4000.pkl")

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5
0,5505853413,4616286686,False,1594307825,1918,3529,106,121,23,15,30,14,11,107,6,60
1,5505853502,4616296280,True,1594307830,2370,3268,54,90,114,69,104,5,113,25,75,70
2,5505853601,4616292509,True,1594307835,2211,3765,28,85,39,83,67,54,9,34,86,63
3,5505853612,4616300387,True,1594307831,2598,3224,36,5,35,27,93,32,84,64,106,21
4,5505853705,4616280855,True,1594307840,1601,3866,49,52,81,14,39,30,36,53,114,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5513436007,4623073549,False,1594771127,2035,3911,82,8,26,51,55,121,14,41,1,108
99996,5513436109,4623074458,False,1594771139,2018,3442,10,107,15,9,119,95,45,25,3,120
99997,5513436512,4623090716,False,1594771154,2635,3223,100,27,4,99,44,111,5,96,42,87
99998,5513436913,4623081152,False,1594771169,2254,3031,69,63,84,7,44,2,45,22,27,73


In [18]:
pd.read_pickle("../data/df_matches_no_limit.pkl")

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5
0,5510478612,4620417806,True,1594586690,2721,3174,7,70,8,30,105,119,73,90,31,18
1,5510478615,4620399961,False,1594586691,1155,3623,86,49,84,75,57,40,120,30,1,60
2,5510478700,4620425526,False,1594586691,3281,3243,108,71,8,26,47,59,74,30,21,14
3,5510478702,4620421211,True,1594586695,2988,2405,34,8,10,86,53,48,18,25,85,75
4,5510478708,4620414900,True,1594586700,2277,4464,67,22,37,107,14,74,7,84,45,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5513436913,4623081152,False,1594771169,2254,3031,69,63,84,7,44,2,45,22,27,73
99996,5513436918,4623070259,True,1594771180,1688,5019,86,53,1,19,84,104,39,113,26,54
99997,5513437003,4623069709,False,1594771184,1790,864,15,108,21,30,53,126,104,110,4,84
99998,5513437208,4623072527,True,1594771189,1928,4327,27,93,6,84,81,10,67,120,101,104


In [19]:
pd.read_pickle("../data/2000_3000_synergy_pivot.pkl")

hero_2,1,2,3,4,5,6,7,8,9,10,...,111,112,113,114,119,120,121,126,128,129
hero_1,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.57,0.59,0.57,0.57,0.55,0.54,0.54,0.58,0.56,...,0.50,0.55,0.61,0.50,0.55,0.53,0.53,0.56,0.53,0.47
2,0.57,NaN,0.51,0.54,0.55,0.55,0.49,0.50,0.57,0.50,...,0.57,0.49,0.44,0.49,0.51,0.52,0.48,0.51,0.53,0.45
3,0.59,0.51,NaN,0.53,0.50,0.56,0.58,0.55,0.57,0.55,...,0.36,0.47,0.51,0.47,0.49,0.54,0.46,0.54,0.53,0.44
4,0.57,0.54,0.53,NaN,0.59,0.51,0.54,0.55,0.55,0.53,...,0.52,0.51,0.51,0.45,0.53,0.50,0.55,0.53,0.44,0.54
5,0.57,0.55,0.50,0.59,NaN,0.52,0.49,0.51,0.56,0.53,...,0.48,0.53,0.50,0.48,0.44,0.54,0.48,0.51,0.49,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.53,0.52,0.54,0.50,0.54,0.46,0.47,0.48,0.53,0.50,...,0.46,0.52,0.47,0.42,0.51,NaN,0.46,0.53,0.45,0.47
121,0.53,0.48,0.46,0.55,0.48,0.46,0.43,0.46,0.46,0.45,...,0.34,0.45,0.55,0.41,0.47,0.46,NaN,0.48,0.40,0.44
126,0.56,0.51,0.54,0.53,0.51,0.49,0.52,0.52,0.54,0.48,...,0.36,0.49,0.45,0.41,0.41,0.53,0.48,NaN,0.53,0.46


In [20]:
pd.read_pickle("../data/2000_3000_counter_pivot.pkl")

hero_2,1,2,3,4,5,6,7,8,9,10,...,111,112,113,114,119,120,121,126,128,129
hero_1,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.50,0.51,0.49,0.54,0.54,0.53,0.56,0.52,0.59,...,0.63,0.60,0.58,0.55,0.55,0.57,0.57,0.57,0.59,0.62
2,0.50,NaN,0.45,0.51,0.51,0.55,0.54,0.54,0.53,0.58,...,0.56,0.58,0.52,0.51,0.54,0.53,0.59,0.52,0.51,0.61
3,0.49,0.55,NaN,0.45,0.49,0.52,0.50,0.55,0.53,0.57,...,0.54,0.54,0.49,0.59,0.61,0.59,0.61,0.52,0.53,0.57
4,0.51,0.49,0.55,NaN,0.52,0.54,0.53,0.49,0.49,0.55,...,0.62,0.50,0.56,0.60,0.56,0.59,0.58,0.57,0.53,0.58
5,0.46,0.49,0.51,0.48,NaN,0.53,0.53,0.51,0.49,0.52,...,0.57,0.56,0.50,0.56,0.50,0.54,0.54,0.49,0.58,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.43,0.47,0.41,0.41,0.46,0.52,0.49,0.46,0.46,0.47,...,0.58,0.48,0.54,0.57,0.51,NaN,0.52,0.51,0.54,0.52
121,0.42,0.41,0.39,0.42,0.46,0.47,0.49,0.45,0.40,0.44,...,0.50,0.49,0.38,0.51,0.47,0.48,NaN,0.42,0.47,0.49
126,0.43,0.48,0.48,0.43,0.51,0.53,0.51,0.48,0.48,0.51,...,0.44,0.55,0.37,0.54,0.50,0.49,0.58,NaN,0.46,0.57


In [21]:
pd.read_pickle("../data/3000_4000_synergy_pivot.pkl")

hero_2,1,2,3,4,5,6,7,8,9,10,...,111,112,113,114,119,120,121,126,128,129
hero_1,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.55,0.60,0.57,0.55,0.54,0.56,0.46,0.59,0.54,...,0.52,0.55,0.51,0.53,0.56,0.53,0.51,0.55,0.56,0.48
2,0.55,NaN,0.58,0.54,0.53,0.54,0.46,0.52,0.55,0.49,...,0.52,0.55,0.54,0.47,0.54,0.55,0.46,0.50,0.54,0.61
3,0.60,0.58,NaN,0.58,0.52,0.57,0.55,0.52,0.57,0.55,...,0.50,0.46,0.48,0.51,0.59,0.54,0.46,0.54,0.64,0.51
4,0.57,0.54,0.58,NaN,0.56,0.60,0.51,0.57,0.57,0.56,...,0.52,0.49,0.36,0.49,0.52,0.48,0.49,0.56,0.53,0.48
5,0.55,0.53,0.52,0.56,NaN,0.51,0.52,0.50,0.51,0.47,...,0.50,0.44,0.50,0.46,0.46,0.52,0.47,0.49,0.48,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.53,0.55,0.54,0.48,0.52,0.49,0.48,0.50,0.51,0.46,...,0.48,0.42,0.49,0.44,0.45,NaN,0.49,0.52,0.47,0.42
121,0.51,0.46,0.46,0.49,0.47,0.39,0.47,0.47,0.45,0.46,...,0.33,0.43,0.44,0.44,0.50,0.49,NaN,0.50,0.42,0.39
126,0.55,0.50,0.54,0.56,0.49,0.48,0.51,0.51,0.54,0.51,...,0.45,0.52,0.55,0.44,0.56,0.52,0.50,NaN,0.53,0.45


In [22]:
pd.read_pickle("../data/3000_4000_counter_pivot.pkl")

hero_2,1,2,3,4,5,6,7,8,9,10,...,111,112,113,114,119,120,121,126,128,129
hero_1,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.47,0.53,0.50,0.52,0.54,0.53,0.58,0.52,0.59,...,0.59,0.56,0.62,0.58,0.54,0.53,0.61,0.56,0.58,0.61
2,0.53,NaN,0.49,0.51,0.53,0.56,0.54,0.51,0.48,0.58,...,0.52,0.57,0.51,0.52,0.49,0.56,0.54,0.53,0.54,0.59
3,0.47,0.51,NaN,0.56,0.53,0.52,0.55,0.62,0.50,0.62,...,0.54,0.54,0.54,0.61,0.58,0.57,0.65,0.54,0.54,0.62
4,0.50,0.49,0.44,NaN,0.57,0.48,0.53,0.52,0.49,0.56,...,0.51,0.58,0.52,0.61,0.56,0.57,0.52,0.57,0.51,0.60
5,0.48,0.47,0.47,0.43,NaN,0.50,0.49,0.50,0.49,0.50,...,0.57,0.53,0.46,0.53,0.48,0.50,0.55,0.48,0.49,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.47,0.44,0.43,0.43,0.50,0.50,0.50,0.50,0.45,0.52,...,0.52,0.52,0.51,0.57,0.47,NaN,0.53,0.46,0.46,0.54
121,0.39,0.46,0.35,0.48,0.45,0.45,0.47,0.45,0.43,0.49,...,0.50,0.48,0.49,0.48,0.40,0.47,NaN,0.45,0.44,0.50
126,0.44,0.47,0.46,0.43,0.52,0.52,0.50,0.55,0.51,0.55,...,0.48,0.51,0.56,0.55,0.49,0.54,0.55,NaN,0.50,0.56


In [23]:
pd.read_pickle("../data/no_limit_synergy_pivot.pkl")

hero_2,1,2,3,4,5,6,7,8,9,10,...,111,112,113,114,119,120,121,126,128,129
hero_1,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.55,0.60,0.56,0.57,0.53,0.55,0.53,0.55,0.52,...,0.52,0.56,0.62,0.54,0.56,0.55,0.52,0.53,0.53,0.49
2,0.55,NaN,0.54,0.57,0.54,0.54,0.48,0.51,0.54,0.49,...,0.50,0.50,0.58,0.52,0.51,0.48,0.45,0.52,0.55,0.47
3,0.60,0.54,NaN,0.55,0.47,0.58,0.55,0.54,0.56,0.51,...,0.38,0.49,0.49,0.49,0.62,0.54,0.49,0.56,0.56,0.50
4,0.56,0.57,0.55,NaN,0.57,0.50,0.51,0.54,0.56,0.48,...,0.55,0.57,0.43,0.44,0.56,0.50,0.51,0.55,0.47,0.49
5,0.57,0.54,0.47,0.57,NaN,0.52,0.47,0.51,0.54,0.49,...,0.51,0.52,0.52,0.50,0.48,0.50,0.46,0.51,0.46,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.55,0.48,0.54,0.50,0.50,0.47,0.49,0.49,0.50,0.45,...,0.49,0.47,0.42,0.43,0.48,NaN,0.49,0.48,0.50,0.41
121,0.52,0.45,0.49,0.51,0.46,0.44,0.47,0.45,0.44,0.46,...,0.37,0.42,0.53,0.44,0.46,0.49,NaN,0.47,0.45,0.40
126,0.53,0.52,0.56,0.55,0.51,0.51,0.50,0.51,0.54,0.44,...,0.41,0.49,0.51,0.48,0.49,0.48,0.47,NaN,0.50,0.50


In [24]:
pd.read_pickle("../data/no_limit_counter_pivot.pkl")

hero_2,1,2,3,4,5,6,7,8,9,10,...,111,112,113,114,119,120,121,126,128,129
hero_1,,,,,,,,,,,,,,,,,,,,,
1,NaN,0.49,0.52,0.51,0.55,0.55,0.53,0.57,0.53,0.60,...,0.63,0.55,0.67,0.56,0.54,0.56,0.58,0.55,0.58,0.61
2,0.51,NaN,0.50,0.51,0.53,0.55,0.53,0.50,0.49,0.59,...,0.53,0.53,0.58,0.52,0.52,0.52,0.56,0.53,0.50,0.59
3,0.48,0.50,NaN,0.50,0.51,0.50,0.51,0.55,0.50,0.64,...,0.55,0.53,0.54,0.55,0.59,0.59,0.65,0.52,0.58,0.59
4,0.49,0.49,0.50,NaN,0.54,0.52,0.54,0.50,0.52,0.56,...,0.52,0.53,0.54,0.58,0.54,0.56,0.53,0.56,0.53,0.64
5,0.45,0.47,0.49,0.46,NaN,0.51,0.54,0.50,0.49,0.52,...,0.53,0.53,0.52,0.55,0.48,0.53,0.54,0.48,0.51,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.44,0.48,0.41,0.44,0.47,0.53,0.46,0.47,0.47,0.48,...,0.54,0.52,0.56,0.56,0.48,NaN,0.51,0.51,0.49,0.56
121,0.42,0.44,0.35,0.47,0.46,0.47,0.47,0.45,0.41,0.49,...,0.51,0.46,0.40,0.50,0.42,0.49,NaN,0.45,0.47,0.48
126,0.45,0.47,0.48,0.44,0.52,0.52,0.53,0.51,0.49,0.56,...,0.47,0.46,0.49,0.57,0.47,0.49,0.55,NaN,0.52,0.54
